In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

<img src="../../../common/logo_DH.svg" align='left' width=50%/>

### Desafío Clustering de Jugadores de Fifa

En esta práctica trabajaremos con un dataset de jugadores de Fifa18. El Fifa es un juego de consola en donde los jugadores de la principales ligas del mundo están descriptos por un conjunto de características como la agilidad, precisión en los tiros libres, etc, además de otros datos como la edad, nacionalidad, salario, y otros sobre su posición en el campo de juego. El dataset está disponible [acá](https://www.kaggle.com/thec03u5/fifa-18-demo-player-dataset). Lamentablemente, contamos con la versión del 2018 en donde pusieron a Cristiano Ronaldo en la tapa, pero cabe mencionar 

Algunas features del dataset son las siguientes:

* Name
* Age 
* Nationality
* Club
* Value: Precio de mercado del jugador
* Wage: Salario del jugador

(...)

* Acceleration
* Agility     
* Balance     
* Ball control
* Free kick accuracy

(...)

* CAM
* CB
* CDM
* CF
* CM                 

Estas últimas codifican numéricamente cuán bien se desenvuelven en las distintas posiciones del campo

<img src='img/field_positions.jpg' width=80%>



<b>El objetivo</b> de esta práctica es ver si hay alguna estructura de clusters en los datos y luego inferir qué criterio es el que mejor los agrupa.

Empezamos levantando el dataset.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('../Data/Fifa_players.csv')

###  Limpieza

* ¿Qué tipo de datos tiene el dataset?

* Conviertan a numérica la columna 'Valor' eliminando el caracter "€" y transformando las M en millones y K en miles.

* Eliminen los jugadores con valor = 0. 

Vamos a implementar un algoritmo de clustering sobre la porción de las features que describen las habilidades de los jugadores. 

* Creen un nuevo dataframe seleccionando las columnas

['Acceleration', 'Aggression', 'Agility', 'Balance',
       'Ball control', 'Composure', 'Crossing', 'Curve', 'Dribbling',
       'Finishing', 'Free kick accuracy', 'GK diving', 'GK handling',
       'GK kicking', 'GK positioning', 'GK reflexes', 'Heading accuracy',
       'Interceptions', 'Jumping', 'Long passing', 'Long shots', 'Marking',
       'Penalties', 'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys']
       
* Conviertan las columnas seleccionadas a tipo numérico, eliminando las filas que sean conflictivas, por ejemplo entradas como '50+8'.       
       
* <b>Importante:</b>No pisen el dataframe original y conserven la relación de índices entre ambos pues vamos a querer recuperar información del dataframe completo para evaluar la composicion de los clusters. 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
     display(data.head(3))
     display(data.dtypes)

In [ ]:
# Tiro algunas columnas 
data=data.drop(['Unnamed: 0','Photo','Flag','Club Logo'],axis=1)

In [ ]:
# Prefijo en el campo Value y Wage
print(data['Value'].apply(lambda x: x[0]).value_counts())
print(data['Wage'].apply(lambda x: x[0]).value_counts())

In [ ]:
# Terminaciones del string en el campo valores
data['Value'].apply(lambda x: x[-1]).value_counts()

In [ ]:
data['Wage'].apply(lambda x: x[-1]).value_counts()

In [ ]:
# Limpio los campos 'Value' y 'Wage'
data['Value']=data['Value'].str.replace('€','');
M_mask=data['Value'].apply(lambda x: x[-1]=='M');
K_mask=data['Value'].apply(lambda x: x[-1]=='K');
#cero_mask=data['Value'].apply(lambda x: x[-1]=='0');

data.loc[M_mask,'Value']=data.loc[M_mask,'Value'].apply(lambda x: np.float(x[:-1])*1e6)
data.loc[K_mask,'Value']=data.loc[K_mask,'Value'].apply(lambda x: np.float(x[:-1])*1e3)
data['Value']=data['Value'].astype(float)

In [ ]:
data['Wage']=data['Wage'].str.replace('€','');
K_mask=data['Wage'].apply(lambda x: x[-1]=='K');
#cero_mask=data['Wage'].apply(lambda x: x[-1]=='0');

data.loc[K_mask,'Wage']=data.loc[K_mask,'Wage'].apply(lambda x: np.float(x[:-1])*1e3)
data['Wage']=data['Wage'].astype(float)

In [ ]:
# Eliminamos los jugadores con valor = 0

data=data.drop(data[data['Value']==0].index,axis=0);


In [ ]:
# Preparamos el dataset para clustering

data_clustering=data[['Acceleration', 'Aggression', 'Agility', 'Balance',
       'Ball control', 'Composure', 'Crossing', 'Curve', 'Dribbling',
       'Finishing', 'Free kick accuracy', 'GK diving', 'GK handling',
       'GK kicking', 'GK positioning', 'GK reflexes', 'Heading accuracy',
       'Interceptions', 'Jumping', 'Long passing', 'Long shots', 'Marking',
       'Penalties', 'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys']]

In [ ]:
# Convertimos las columnas a numerico y eliminamos las filas conflictivas

object_columns=[col for col in data_clustering.columns if data[col].dtype=='object']

mask=data_clustering[object_columns].applymap(lambda x:  not str(x).isnumeric())
data_clustering=data_clustering.drop(data_clustering[mask.sum(axis=1)>0].index,axis=0)


### Clustering


* Implementen un algoritmo de KMeans variando el número de clusters en un rango que les suene razonable. 

<b>¿Qué hay que hacer antes con los datos?</b>


* Grafiquen la inercia (suma de distancias cuadráticas a los centroides) en función de k. Esta medida queda almacenada en la propiedad 'inertia_' del objeto KMeans una vez entrenado.

* Siguiendo la regla del codo, hay un valor de k preferencial?



In [ ]:
scaler=StandardScaler()
X=scaler.fit_transform(data_clustering)

inertia=[]
sil=[]
ch_scores=[]
k_values=range(2,20,2)

for k in k_values:
    print(k)
    km=KMeans(n_clusters=k)
    km.fit(X)
    inertia.append(km.inertia_)    

In [ ]:
plt.plot(k_values,inertia);
plt.vlines(4,np.min(inertia),np.max(inertia),linestyle='dashed');
plt.xlabel('Numero de clusters (k)',fontsize=15);plt.ylabel('Inertia',fontsize='15')

### Composición de los clusters

Veamos cómo están compuestos los clusters, por ejemplo mirando las posiciones preferibles para cada jugador. Esta información está en el campo 'Preferred position' del dataset original.

* Hagan un gráfico de barras para cada cluster mostrando la proporción de jugadores de cada posición que lo componen.

In [ ]:
k=4;
km=KMeans(n_clusters=k,random_state=0)
km.fit(X)

data_filtered=data.loc[data_clustering.index];
f,ax=plt.subplots(2,2,figsize=(15,10),gridspec_kw={'wspace':0.5})
ax=ax.ravel();
for lbl in range(k):
    S=data_filtered.loc[km.labels_==lbl,'Preferred Positions'].value_counts(normalize=True);
    chart=sns.barplot(y=S.index[:10],x=S.values[:10],ax=ax[lbl],orient='h')
    chart.set_title('Cluster '+str(lbl))


### Más clusters

Vimos que los clusters contienen información sobre la posición en el campo. Si forzamos k = 15, 

* ¿se ven las posiciones pintadas de distinto color en la imagen de la introducción?

* Si encuentran clusters con una composición parecida de posiciones, ¿qué variable puede ser relevante para separarlos? Tal vez el valor... Grafiquen simultaneamente la distribución de valores de los jugadores para los clusters de composición similar.

In [ ]:
k=15;
km=KMeans(n_clusters=k,random_state=0)
km.fit(X)

In [ ]:
data_filtered=data.loc[data_clustering.index]
f,ax=plt.subplots(3,5,figsize=(15,10),gridspec_kw={'wspace':0.5})
ax=ax.ravel();
for lbl in range(k):
    S=data_filtered.loc[km.labels_==lbl,'Preferred Positions'].value_counts(normalize=True);
    chart=sns.barplot(y=S.index[:10],x=S.values[:10],ax=ax[lbl],orient='h')
    chart.set_title('Cluster '+str(lbl))    
    #chart.set_xticklabels(chart.get_xticklabels(), rotation=90);

In [ ]:
grupos=[[0,12],[1,2],[3,9,14],[4,5,6,8],[7,11],[10,13]];

f,ax=plt.subplots(3,2,figsize=(15,10),gridspec_kw={'wspace':0.5});
ax=ax.ravel();

for i,g in enumerate(grupos):
    for c in g:        
        values=data_filtered.loc[km.labels_==c,'Value'];                        
        sns.kdeplot(np.log(values),label='Cluster '+str(c),ax=ax[i]);        
    ax[i].legend();
    

### Visualización en un espacio de dimensionalidad reducida

* Reduzcamos la dimensionalidad del dataset usando PCA y conservando dos componentes principales.

* Grafiquemos con un scatterplot los 4 clusters obtenidos originalmente en este espacio reducido.

In [ ]:
k=4;
km=KMeans(n_clusters=k,random_state=0)
km.fit(X)

pca=PCA(n_components=2)
X_transformed=pca.fit_transform(X)

sns.scatterplot(X_transformed[:,0],X_transformed[:,1],hue=km.labels_,palette='dark');
